In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load data

In [ ]:
flowUpper = pd.read_excel("data/ACA_upper_course.xlsx", index_col=0)
flowLower = pd.read_excel("data/ACA_lower_course.xlsx", index_col=0)
rain = pd.read_excel("data/SMC.xlsx", index_col=0)

In [ ]:
rain

## Aligning series

In [ ]:
def alignSeriesIntoDf(df, serie, lag):
    minutesRow = 30
    posToMove = int(lag / minutesRow)  
    vec = serie.to_numpy()
    size = len(vec)
    vec = np.pad(vec, (posToMove,0), 'constant', constant_values=(0))
    vec = vec[:size]
    new_series = serie.copy()
    new_series.iloc[:] = vec
    df = pd.concat([df, new_series], axis=1)
    return df

### Upper course

In [ ]:
dfUpper = pd.DataFrame(data={"Date":pd.date_range("2009-01-01", periods=210385, freq="30T")}).set_index("Date")
dfUpper = alignSeriesIntoDf(dfUpper,  flowUpper["L17147-72-00005"], 210)
dfUpper = alignSeriesIntoDf(dfUpper,  flowUpper["L17167-72-00001"], 330)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["DG"], 360)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["CG"], 360)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["CI"], 360)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["V4"], 420)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["CC"], 570)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["CY"], 450)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["VN"], 300)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["WS"], 510)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["V3"], 420)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["V5"], 330)
dfUpper = alignSeriesIntoDf(dfUpper,  rain["Z4_ZC"], 420)
dfUpper = alignSeriesIntoDf(dfUpper,  flowUpper["L08116-72-00002"], 0) # to predict (dependent attribute)
dfUpper = dfUpper.loc["2009-01-02":] # to remove padded initial '0' values

In [ ]:
dfUpper

### Lower course

In [ ]:
dfLower = pd.DataFrame(data={"Date":pd.date_range("2009-01-01", periods=210385, freq="30T")}).set_index("Date")
dfLower = alignSeriesIntoDf(dfLower,  flowLower["F026458"], 270)
dfLower = alignSeriesIntoDf(dfLower,  flowLower["F001242"], 420)
dfLower = alignSeriesIntoDf(dfLower,  flowLower["L17079-72-00005"], 240)
dfLower = alignSeriesIntoDf(dfLower,  flowLower["L17079-72-00004"], 240)
dfLower = alignSeriesIntoDf(dfLower,  flowLower["L17038-72-00002"], 210)
dfLower = alignSeriesIntoDf(dfLower,  rain["DN"], 690)
dfLower = alignSeriesIntoDf(dfLower,  rain["UO"], 1020)
dfLower = alignSeriesIntoDf(dfLower,  rain["UN"], 810)
dfLower = alignSeriesIntoDf(dfLower,  rain["WS"], 630)
dfLower = alignSeriesIntoDf(dfLower,  rain["VN"], 720)
dfLower = alignSeriesIntoDf(dfLower,  rain["DJ"], 480)
dfLower = alignSeriesIntoDf(dfLower,  rain["DM_XJ_WF"], 660)
dfLower = alignSeriesIntoDf(dfLower,  flowLower["L17055-72-00002"], 0) # to predict (dependent attribute)
dfLower = dfLower.loc["2009-01-02":] # to remove padded initial '0' values

In [ ]:
dfLower

In [ ]:
dfLower.to_csv('data/auxdf/dflower.csv')
dfUpper.to_csv('data/auxdf/dfupper.csv')

## Select subsets of interest

### Select per precipitation
#### Upper course

In [ ]:
# use this number to separate rain from no-rain
th = 0.1


mean_rain = dfUpper[['DG','CG','CI','V4','CC','CY','VN','WS','V3','V5','Z4_ZC']].mean(axis=1)
print(mean_rain.shape)

mean_rain_pos = mean_rain[mean_rain > th]
print(mean_rain_pos.shape)

dfUpper_rain = dfUpper.loc[np.intersect1d(dfUpper.index, mean_rain_pos.index)]
print(dfUpper_rain.shape)

dfUpper_norain = dfUpper.loc[np.setdiff1d(dfUpper.index, mean_rain_pos.index)]
print(dfUpper_norain.shape)

#### Lower course

In [ ]:
mean_rain = dfLower[['UO','UN','WS','DJ','DN','DM_XJ_WF']].mean(axis=1)
print(mean_rain.shape)

mean_rain_pos = mean_rain[mean_rain > th]
print(mean_rain_pos.shape)

dfLower_rain = dfLower.loc[np.intersect1d(dfLower.index, mean_rain_pos.index)]
print(dfLower_rain.shape)

dfLower_norain = dfLower.loc[np.setdiff1d(dfLower.index, mean_rain_pos.index)]
print(dfLower_norain.shape)

In [ ]:
dfLower_norain.reset_index(drop=True, inplace=True)
dfUpper_norain.reset_index(drop=True, inplace=True)

dfLower_norain.to_csv('data/auxdf/dflower_norain.csv',index=False)
dfUpper_norain.to_csv('data/auxdf/dfupper_norain.csv',index=False)

dfLower_rain.reset_index(drop=True, inplace=True)
dfUpper_rain.reset_index(drop=True, inplace=True)

dfLower_rain.to_csv('data/auxdf/dflower_rain.csv',index=False)
dfUpper_rain.to_csv('data/auxdf/dfupper_rain.csv',index=False)

### Upsample precipitation-only subset
#### Upper course

In [ ]:
import smogn

dfUpper_rain_up = dfUpper_rain.copy()
print(dfUpper_rain_up.index)
print('Initial dimensions: ', dfUpper_rain_up.shape)
dfUpper_rain_up = smogn.smoter(
    
    ## main arguments
    data = dfUpper_rain_up,      ## pandas dataframe
    y = 'L08116-72-00002',    ## string ('header name')
    k = 9,                    ## positive integer (k < n)
    samp_method = 'extreme',  ## string ('balance' or 'extreme')
    rel_thres = 0.10
)
dfUpper_rain_up.reset_index(drop=True, inplace=True)
print('Dimensions after upsampling: ', dfUpper_rain_up.shape)

In [ ]:
dfUpper_rain['L08116-72-00002'].plot.kde(label='Original')
dfUpper_rain_up['L08116-72-00002'].plot.kde(label='Upsampled')
plt.legend()
plt.show()

#### Lower course

In [ ]:
dfLower_rain_up = dfLower_rain.copy()
print(dfLower_rain_up.index)
print('Initial dimensions: ', dfLower_rain.shape)
dfLower_rain_up = smogn.smoter(
    
    ## main arguments
    data = dfLower_rain_up,      ## pandas dataframe
    y = 'L17055-72-00002',    ## string ('header name')
    k = 5,                    ## positive integer (k < n)
    samp_method = 'extreme',  ## string ('balance' or 'extreme')
)
dfLower_rain_up.reset_index(drop=True, inplace=True)
print('Dimensions after upsampling: ', dfLower_rain_up.shape)

In [ ]:
dfLower_rain['L17055-72-00002'].plot.kde(label='Original')
dfLower_rain_up['L17055-72-00002'].plot.kde(label='Upsampled')
plt.legend()
plt.show()

In [ ]:
dfLower_rain_up.to_csv('data/auxdf/dflower_rainup.csv',index=False)
dfUpper_rain_up.to_csv('data/auxdf/dfupper_rainup.csv',index=False)

# Learning and testing

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import TimeSeriesSplit, HalvingGridSearchCV
from sklearn.metrics import mean_squared_error
import timeit

def train_and_test(df_results, Xy, real_Xy, clf, possible_params, relevant_periods, CVS=False):
    start = timeit.default_timer()

    tscv = TimeSeriesSplit(n_splits=3)
    *_, (_,csv_ts_idx) = tscv.split(Xy)

    for i, (tr_idx, ts_idx) in enumerate(tscv.split(Xy)):
        print("Validation it.: ", i+1, " Model: ", type(clf).__name__)

        # divide date
        X_train, y_train = Xy.values[tr_idx,:-1], Xy.values[tr_idx,-1]
        X_test, y_test = Xy.values[ts_idx,:-1], Xy.values[ts_idx,-1]
        if CVS:
            X_test, y_test = Xy.values[csv_ts_idx,:-1], Xy.values[csv_ts_idx,-1]

        fit = None
        if not possible_params:
            fit = clf.fit(X_train, y_train)
        else:
            # find optimal parameters
            optimizer = HalvingGridSearchCV(estimator=clf, cv=TimeSeriesSplit(n_splits=3), 
                                            param_grid=possible_params, n_jobs=-1)
            fit = optimizer.fit(X_train, y_train).best_estimator_

        
        hat_y_test = fit.predict(X_test)
        
        score = fit.score(X_test, y_test)
        rmse_all = mean_squared_error(y_test, hat_y_test, squared=False)
        
        result_row = {'Method':type(clf).__name__, 'RMSE':rmse_all, 'Score':score}
        
        for period in relevant_periods:
            X_period_test = real_Xy.loc[relevant_periods[period]].values[:,:-1]
            y_period_test = real_Xy.loc[relevant_periods[period]].values[:,-1]

            hat_y_period_test = fit.predict(X_period_test)
        
            rmse_period = mean_squared_error(y_period_test, hat_y_period_test, squared=False)
            result_row["RMSE_"+period] = rmse_period

            y_period_to_plot = pd.DataFrame(data={"real":y_period_test, 
                                                  "predicted":hat_y_period_test, 
                                                  "date":relevant_periods[period]})
            y_period_to_plot.set_index("date", inplace=True)
            y_period_to_plot.plot(title=period,ylabel="River flow")


        df_results = df_results.append(result_row, ignore_index=True)
        plt.show()


    stop = timeit.default_timer()
    print("Time lapse (seconds): ", (stop-start))
    
    return df_results

In [ ]:
periods_of_interest = {"Standard_1":pd.date_range("2019-05-01 5:00:00","2019-05-11 21:30:00",freq="30T"),
                       "Standard_2":pd.date_range("2020-04-20 5:00:00","2020-04-26 21:30:00",freq="30T"),
                       "Gloria_storm":pd.date_range("2020-01-20 21:30:00","2020-01-25 5:00:00",freq="30T"),
                       "Leslie_storm":pd.date_range("2018-10-14 5:00:00","2018-10-20 21:30:00",freq="30T")}
columns_res = ["Method","Score","RMSE"]
for p in periods_of_interest:
    columns_res.append("RMSE_"+p)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor


types_of_models = {
    RandomForestRegressor(): { 
        'n_estimators': [50,100, 200], 
        'max_features': [1, 'sqrt', 'log2'], 
        'max_depth' : [8,10,25,None] 
    },
    XGBRegressor(): { 
        'n_estimators': [50,100, 200], # Nombre de arbres
        'max_features': [1, 'sqrt', 'log2'], # Maxim features
        'max_depth' : [8,10,25,None] # Profunditat dels arbres
    },
    KNeighborsRegressor(): { 
        'n_neighbors': [1, 3, 5, 10, 20], # Nombre de neighbors
        'leaf_size': [20, 30, 50], # Mida fulles
        'weights': ['uniform','distance']
    },
    LinearRegression(): {}
}

In [ ]:
def run_experiment(name, data_orig, data_transf, columns_res, types_of_models, periods_of_interest, use_cvs):
    df_results = pd.DataFrame(columns=columns_res)
    for model in types_of_models:
        df_results = train_and_test(df_results, data_transf, data_orig, 
                                    model, types_of_models[model], 
                                    periods_of_interest, CVS=use_cvs)
    if use_cvs: name += "_CVS"
    else: name += "_TSS"
    df_results.to_csv("results/"+name+".csv")
    return df_results

## Collect results for the Complete dataset

#### To run one experiment:

In [ ]:
# select type of validation: TSS or CVS
use_cvs = True
# select the course of the river to analyze: "upper" or "lower"
course = "lower"

In [ ]:
data_orig = pd.read_csv('data/auxdf/df'+course+'.csv', index_col=0)
data_orig.index=data_orig.index.astype("datetime64[ns]")
data_transf = data_orig.copy()

In [ ]:
run_experiment(course + "_completeData", data_orig, data_transf, columns_res, 
               types_of_models, periods_of_interest, use_cvs)

#### To run all 4 experiments at once:

In [ ]:
for course in ["lower", "upper"]:
    data_orig = pd.read_csv('data/auxdf/df'+course+'.csv', index_col=0)
    data_orig.index=data_orig.index.astype("datetime64[ns]")
    data_transf = data_orig.copy()
    for use_cvs in [True, False]:
        run_experiment(course + "_completeData", data_orig, data_transf, columns_res, 
                       types_of_models, periods_of_interest, use_cvs)

## Collect results for the No-precipitation subset

#### To run one experiment:

In [ ]:
# select type of validation: TSS or CVS
use_cvs = False
# select the course of the river to analyze: "upper" or "lower"
course = "upper"

In [ ]:
data_orig = pd.read_csv('data/auxdf/df'+course+'.csv', index_col=0)
data_orig.index=data_orig.index.astype("datetime64[ns]")
data_transf = pd.read_csv('data/auxdf/df'+course+'_norain.csv')

In [ ]:
run_experiment(course + "_noPrecipitation", data_orig, data_transf, columns_res, 
               types_of_models, periods_of_interest, use_cvs)

#### To run all 4 experiments at once:

In [ ]:
for course in ["lower", "upper"]:
    data_orig = pd.read_csv('data/auxdf/df'+course+'.csv', index_col=0)
    data_orig.index=data_orig.index.astype("datetime64[ns]")
    data_transf = pd.read_csv('data/auxdf/df'+course+'_norain.csv')
    for use_cvs in [True, False]:
        run_experiment(course + "_noPrecipitation", data_orig, data_transf, columns_res, 
                       types_of_models, periods_of_interest, use_cvs)

## Collect results for the Precipitation-only subset

#### To run one experiment:

In [ ]:
# select type of validation: TSS or CVS
use_cvs = False
# select the course of the river to analyze: "upper" or "lower"
course = "upper"

In [ ]:
data_orig = pd.read_csv('data/auxdf/df'+course+'.csv', index_col=0)
data_orig.index=data_orig.index.astype("datetime64[ns]")
data_transf = pd.read_csv('data/auxdf/df'+course+'_rainup.csv')

In [ ]:
run_experiment(course + "_precipitationOnly", data_orig, data_transf, columns_res, 
               types_of_models, periods_of_interest, use_cvs)

#### To run all 4 experiments at once:

In [ ]:
for course in ["lower", "upper"]:
    data_orig = pd.read_csv('data/auxdf/df'+course+'.csv', index_col=0)
    data_orig.index=data_orig.index.astype("datetime64[ns]")
    data_transf = pd.read_csv('data/auxdf/df'+course+'_rainup.csv')
    for use_cvs in [True, False]:
        run_experiment(course + "_precipitationOnly", data_orig, data_transf, columns_res, 
                       types_of_models, periods_of_interest, use_cvs)